# Point Query and Zonal Statistics for Data #

In [56]:
import pandas as pd
import re
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterstats
import os
import numpy as np
from rasterstats import zonal_stats # gives us raster summaries within a shapefile
from rasterstats import point_query # gives us raster summaries within a shapefile
from shapely.geometry import Polygon, Point
import rasterio
import shapely.speedups
shapely.speedups.enable()

In [57]:
#os.chdir('../../')
#os.chdir('../')
#os.chdir("Analysis")
os.getcwd()

'C:\\Users\\lgorman\\OneDrive\\Desktop\\PhD\\Analysis'

In [58]:
rasters_to_attach={"education":os.path.join("data", "raster","ClimAfr21_education_index","ClimAfr21_education index.tif"),
                  "technological_capital":os.path.join("data", "raster","ClimAfr08_technological_capital_index","ClimAfr08_technological capital index.tif"),
                  "infrastructure":os.path.join("data", "raster","ClimAfr23_infrastructure_index","ClimAfr23_infrastructure index.tif"),
                  "conflictuality":os.path.join("data", "raster","ClimAfr27_conflictuality_index","ClimAfr27_conflictuality index.tif"),
                  #"soil":os.path.join("data", "raster","HWSD_RASTER","hwsd.bil"),
                  "cattle_density":os.path.join("data", "raster","cattle_density_map","Cattle10km_AD_2010_v2_1.tif"),
                  "Clim_Afr_AEZ":os.path.join("data", "raster","ClimAfr_AEZ","ClimAf_1_1981_2050_lpjm_mir5_B1_AEZAI_ann_avgCUR05_10km.tif"),
                  "financial_capital":os.path.join("data", "raster","ClimAfr09_financial_capital_index","ClimAfr09_financial capital index.tif"),
                  "rurality":os.path.join("data", "raster","ClimAfr13_rurality_index","ClimAfr13_rurality index.tif"),
                  "gender_gap":os.path.join("data", "raster","ClimAfr15_gender_gap_index","ClimAfr15_gender gap index.tif"),
                  "household_technological_capital":os.path.join("data", "raster","ClimAfr22_household_technology_index","ClimAfr22_household technology index.tif"),
                  "financial_development":os.path.join("data", "raster","ClimAfr24_financial_development_index","ClimAfr24_financial development index.tif"),
                  "richness":os.path.join("data", "raster","ClimAfr25_richness_index","ClimAfr25_richness index.tif"),
                  "governance":os.path.join("data", "raster","ClimAfr26_governance_index","ClimAfr26_governance index.tif"),
                  #"slope":os.path.join("data", "raster","global_slope_median","plate09.bil"),
                  "population_density":os.path.join("data", "raster","gridded_pop_world_population_density_2015","gpw_v4_population_density_adjusted_to_2015_unwpp_country_totals_rev11_2015_2pt5_min.tif"),
                  "institutional_capital":os.path.join("data", "raster","ClimAfr10_institutional_capital_index","ClimAfr10_institutional capital index.tif"),
                  "AEZ":os.path.join("data", "raster","AEZ_2009","AEZ_FINAL.asc"),
                  "time_to_market_20_k":os.path.join("data", "raster","Harvest_Choice_Market_Travel","TravelTimeToMarket_SSA_GeoTiff","traveltimetomarket_ssa_020k.tif"),
                  "time_to_market_50_k":os.path.join("data", "raster","Harvest_Choice_Market_Travel","TravelTimeToMarket_SSA_GeoTiff","traveltimetomarket_ssa_050k.tif"),
                  "time_to_market_100_k":os.path.join("data", "raster","Harvest_Choice_Market_Travel","TravelTimeToMarket_SSA_GeoTiff","traveltimetomarket_ssa_100k.tif"),
                  "time_to_market_250_k":os.path.join("data", "raster","Harvest_Choice_Market_Travel","TravelTimeToMarket_SSA_GeoTiff","traveltimetomarket_ssa_250k.tif"),
                  "time_to_market_500_k":os.path.join("data", "raster","Harvest_Choice_Market_Travel","TravelTimeToMarket_SSA_GeoTiff","traveltimetomarket_ssa_500k.tif")}
                  #"pests_and_disease":os.path.join("data", "raster","Harvest_Choice_Pests_and_Disease","Pests_and_Disease.tiff")}

raster_keys=[value for value, key in  rasters_to_attach.items()]
raster_labels=[key for value, key in  rasters_to_attach.items()]


In [76]:
AEZ_mapping_categories={101:"Temperate / arid",
                    102:"Temperate / Semi-arid",
                    103:"Temperate / sub-humid",
                    104:"Temperate / humid",
                    211:"Subtropic - warm / arid",
                    212:"Subtropic - warm / semiarid",
                    213:"Subtropic - warm / subhumid",
                    214:"Subtropic - warm / humid",
                    221:"Subtropic - cool / arid",
                    222:"Subtropic - cool / semiarid",
                    223:"Subtropic - cool / subhumid",
                    224:"Subtropic - cool / humid",
                    311:"Tropic - warm / arid",
                    312:"Tropic - warm / semiarid",
                    313:"Tropic - warm / subhumid",
                    314:"Tropic - warm / humid",
                    321:"Tropic - cool / arid",
                    322:"Tropic - cool / semiarid",
                    323:"Tropic - cool / subhumid",
                    324:"Tropic - cool / humid",
                    400:"Boreal"}


In [86]:
AEZ_values=[index for index in AEZ_mapping_categories.values()]
AEZ_keys=[index for index in AEZ_mapping_categories.keys()]

pd.DataFrame({"AEZ_Names":AEZ_values,"AEZ_Numbers":AEZ_keys}).to_csv(os.path.join("data","AEZ_Categories","AEZ_categories.csv" ))

In [60]:
def extract_point_values(filepath,datatype, point_data):
    with rasterio.open(filepath) as src:
        # can check the profile and type of src, but effectively is an open file command
        affine = src.transform # transforms the raster dataset into the coordinate reference system
        
        raster_array=rasterio.open(filepath).read(1) # why is this "1" here?
        
        if datatype!="AEZ":
            variable_AEZ=0
        elif datatype!="Clim_Afr_AEZ":
            variable_AEZ=0
        elif datatype=="AEZ":
            variable_AEZ=1
        elif datatype=="Clim_Afr_AEZ":
            variable_AEZ=1
        if variable_AEZ==0:
            raster_array=raster_array.astype(float)
        #raster_array[raster_array<-3.4e+38]=np.nan
            raster_array[raster_array<-2e+9]=np.nan
            nodata=np.nan
        if variable_AEZ==1:
            raster_array=raster_array.astype(int)
            nodata=int(-9999)

        point_query_data = pd.DataFrame(point_query(point_data,raster_array ,affine=affine, nodata=nodata, interpolate='nearest')) # calculates raster statistics within boundary of shapefile
        raster_array=None
    point_query_data=pd.concat([point_data,point_query_data],axis=1)
    point_query_data.columns= ["raster_level_" + datatype if column==0 else column for column in point_query_data.columns]    
    return point_query_data;

In [61]:
def create_geo_data_frame_from_point_dataset (dataframe, crs, name_of_lat_column, name_of_lon_column):
    latitude_not_null=dataframe.loc[:,name_of_lat_column].notnull()
    longitude_not_null=dataframe.loc[:,name_of_lon_column].notnull()
    dataframe=dataframe.loc[latitude_not_null&longitude_not_null,:]
    
    geometry=[Point(xy) for xy in zip(dataframe[name_of_lon_column], dataframe[name_of_lat_column])]
    dataframe = gpd.GeoDataFrame(dataframe, geometry=geometry,crs=crs)
    return dataframe;

# defining a coordinate reference system 
crs={'init' :'epsg:4326'}


points_rhomis_geodataframe=pd.read_csv(os.path.join("data","rhomis_data","RHoMIS_Indicators.csv"), encoding="latin1", low_memory=False)
#points_rhomis_geodataframe=pd.read_csv(os.path.join("data","processed","rhomis_points_with_country_level.csv"), encoding="latin1", low_memory=False)
name_of_lat_column="GPS_LAT"
name_of_lon_column="GPS_LON"

latitude_not_null=points_rhomis_geodataframe.loc[:,"GPS_LAT"].notnull()
longitude_not_null=points_rhomis_geodataframe.loc[:,"GPS_LON"].notnull()
points_rhomis_geodataframe=points_rhomis_geodataframe.loc[latitude_not_null&longitude_not_null,:]

points_rhomis_geodataframe=create_geo_data_frame_from_point_dataset(dataframe=points_rhomis_geodataframe,
                                                                   crs=crs,
                                                                   name_of_lat_column=name_of_lat_column,
                                                                   name_of_lon_column=name_of_lon_column)

world_geopanda = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) #continents dataframe
africa_shapefile=world_geopanda.loc[world_geopanda["continent"]=="Africa", :].envelope
#africa_shapefile_index= [index for index in range(africa_shapefile.shape[0])]
#africa_shapefile=gpd.GeoSeries(africa_shapefile, index=africa_shapefile_index)


#points_rhomis_geodataframe.within(world_geopanda.loc[world_geopanda["continent"]=="Africa","geometry"]).
world_geopanda.columns
#points_rhomis_geodataframe.within(africa_shapefile)
point_in_envelope=np.zeros(points_rhomis_geodataframe.shape[0], dtype=bool)
#list(africa_shapefile.index)
#gpd.GeoSeries(africa_shapefile.iloc[:,"0"])
points_rhomis_geodataframe.reset_index(drop=True, inplace=True)
africa_shapefile.reset_index(drop=True, inplace=True)
africa_shapefile=gpd.GeoDataFrame(africa_shapefile, columns=["geometry"])
for points in points_rhomis_geodataframe.index:
    temp_point=points_rhomis_geodataframe.loc[points,"geometry"]
    for polygon in africa_shapefile.index:
        if temp_point.within(africa_shapefile.loc[polygon,"geometry"])==True:
            point_in_envelope[points]=True
rhomis_in_africa=points_rhomis_geodataframe.loc[point_in_envelope,:]
rhomis_in_africa.reset_index(drop=True, inplace=True)

In [13]:
#polygon=1
#point



In [62]:
point_data_old=rhomis_in_africa
#points_rhomis_geodataframe=None
for loop_index in range(len(rasters_to_attach)):
    print(raster_keys[loop_index])
    point_data_new=extract_point_values(filepath=raster_labels[loop_index], datatype=raster_keys[loop_index], point_data=point_data_old)
    point_data_old=point_data_new
processed_file_name="point_data_"+"with_raster_values.csv"
point_data_old.to_csv(os.path.join("data","processed",processed_file_name))

education
technological_capital
infrastructure
conflictuality
cattle_density
Clim_Afr_AEZ
financial_capital
rurality
gender_gap
household_technological_capital
financial_development
richness
governance
population_density
institutional_capital
AEZ
time_to_market_20_k
time_to_market_50_k
time_to_market_100_k
time_to_market_250_k
time_to_market_500_k


In [63]:
raster_level_columns=["raster_level_"]

all_columns=point_data_old.columns
def find_and_return_existing_patterns(pattern,item_to_check):
    test=re.search(pattern,item_to_check)
    if test is not None:
        return item_to_check;
    if test is None:
        return None;

def grouping_pattern_in_list(pattern, list_to_check):
    list_of_matches=[]
    [list_of_matches.append(find_and_return_existing_patterns(pattern=pattern,item_to_check=item_to_check))  for item_to_check in list_to_check  if find_and_return_existing_patterns(pattern=pattern,item_to_check=item_to_check) is not None]
    return list_of_matches;

def grouping_multiple_patterns_in_list(list_of_patterns, list_to_check):
    list_of_matches=[]
    [list_of_matches.append(grouping_pattern_in_list(pattern=pattern, list_to_check=list_to_check))  for pattern in list_of_patterns]
    unnested_list=[]
    
    for sublist in range(len(list_of_matches)):
        temp_list=list_of_matches[sublist]
        for item_in_sublist in temp_list:
            unnested_list.append(item_in_sublist)        
    return unnested_list;


raster_columns=grouping_multiple_patterns_in_list(list_of_patterns=raster_level_columns, list_to_check=all_columns)
columns_in_raster=list(map(lambda x: x in raster_columns, list(point_data_old.columns)))


In [64]:
raster_data_temp=point_data_old.loc[:,columns_in_raster]

In [73]:
raster_data_temp.isna().any(axis=1).value_counts() #checking how many rows have na values

rows_to_keep=raster_data_temp.isna().any(axis=1)==False

In [74]:
no_raster_nas=point_data_old.loc[rows_to_keep,:]
no_raster_nas.shape

(13742, 75)

In [75]:
processed_file_name="point_data_"+"with_raster_values.csv"
no_raster_nas.to_csv(os.path.join("data","processed",processed_file_name))